In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install libraries, import them and load data

In [2]:
! pip install  plotly dash dash-bootstrap-components


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import libraries
import dash
import dash_bootstrap_components as dbc
from dash import dcc, Input, Output,html      # dcc: dash core component, input and output: could be used for 'callbacks'
import plotly.express as px #  for graph
import pandas as pd

In [46]:
# Loading data
data= pd.read_csv('healthcare.csv')
data.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [47]:
def load_data():
  data=pd.read_csv('healthcare.csv')
  data['Billing Amount']=pd.to_numeric(data['Billing Amount'],errors='coerce')
  data['Date of Admission']=pd.to_datetime(data['Date of Admission'])
  data['YearMonth']=data['Date of Admission'].dt.to_period('M')
  return data
data=load_data()


In [48]:
data.shape

(55500, 16)

In [49]:
data.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,YearMonth
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2024-01
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,2019-08
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,2022-09
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,2020-11
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,2022-09


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                55500 non-null  object        
 1   Age                 55500 non-null  int64         
 2   Gender              55500 non-null  object        
 3   Blood Type          55500 non-null  object        
 4   Medical Condition   55500 non-null  object        
 5   Date of Admission   55500 non-null  datetime64[ns]
 6   Doctor              55500 non-null  object        
 7   Hospital            55500 non-null  object        
 8   Insurance Provider  55500 non-null  object        
 9   Billing Amount      55500 non-null  float64       
 10  Room Number         55500 non-null  int64         
 11  Admission Type      55500 non-null  object        
 12  Discharge Date      55500 non-null  object        
 13  Medication          55500 non-null  object    

In [51]:
num_records=len(data)
avg_billing=data['Billing Amount'].mean()

In [52]:
len(data)

55500

# Creating a web application

In [53]:
app=dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App Layout and Design

In [54]:
from logging import debug
app=dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    # Title
    dbc.Row([
        dbc.Col([
            html.H1("Healthcare Dashboard")
        ], width=12, className="text-center my-5")
    ]),
    # Hospital Statistics
    dbc.Row([
        dbc.Col(html.Div(f"Total Patient Record: {num_records}",className="text-center my-5 top-text"),width=7),# my-5 means margin =5
        dbc.Col(html.Div(f'Average Billing Amount: {avg_billing:,.2f}',className="text-center my-5 top-text"),width=7)
    ],class_name='mb-5'),# mb-5 means 'margin bottom=5'

    # Male and Female Patient Demographic
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4('Patient Demographics',className='card-title'),
                    dcc.Dropdown(id='gender-filter',
                                 options=[{'label':gender,'value':gender} for gender in data['Gender'].unique()],
                                 value=None,
                                 placeholder='Select a gender'),
                    dcc.Graph(id='age-distribution')
                ])
            ]) # 'dbc.Card' holds data 
        ],width=7),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4('Medical Condition Distribution',className='card-title'),
                    dcc.Graph(id='Condition-distribution')
                ])
            ]) # 'dbc.Card' holds data 
        ],width=7)
    ]),
    # Insurance Provider Data
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4('Insurance Provider Comparison',className='card-title'),
                    dcc.Graph(id='insurance-comparison')
                ])
            ])
        ],width=12)
    ]),

    # Billing Distribution
    dbc.Row([
        dbc.Col([
            dbc.CardBody([
                html.H4('Billing Amount Distribution',className='card-title'),
                dcc.Slider(id='billing-slider',
                           min=data['Billing Amount'].min(),
                           max=data['Billing Amount'].max(),
                           value=data['Billing Amount'].median(),
                           marks={int(value):f"${int(value):,}" for value in data['Billing Amount'].quantile([0,0.25,0.5,0.75,1]).values},
                           step=100),
                dcc.Graph(id='billing-distribution')
            ])
        ],width=12)
    ]),
    # Trends in Admission
    dbc.Row([
        dbc.Col([
            dbc.CardBody([
                html.H4('Trends in Admission',className='card-title'),
                dcc.RadioItems(id='chart-type',
                               options=[{'label':'Line Chart','value':'line'},{'label':'Bar Chart','value':'Bar'}],
                               value='line',
                               inline=True,
                               className='mb-4'),
                dcc.Dropdown(id='condition-filter',
                             options=[{'label':condition,'value':condition} for condition in data['Medical Condition'].unique()],
                             value=None,
                             placeholder='Select a condition'),
                dcc.Graph(id='admission-trends')
            ])
        ],width=12)
    ]),

],fluid=True)

# Creating our Callback
#------------------------------
# 1. Callback for Male and Female Patient Demographic
#@app.route('/')# while we are using Flask 'we create path'
@app.callback(
    Output('age-distribution','figure'),
    Input('gender-filter','value')
)
def update_distribution(selected_gender):
    if selected_gender:
        filtered_data=data[data['Gender']== selected_gender]
    else:
        filtered_data=data
    if filtered_data.empty:
        return {}
    fig=px.histogram(
        filtered_data,
        x='Age',
        nbins=10,
        color='Gender',
        title='Age Distribution by Gender',
        color_discrete_sequence=['#636EFA','#EF553B']
    )
    return fig
# 2 callback for Medical condition distribution
@app.callback(
    Output('Condition-distribution','figure'),
    Input('gender-filter','value')
)
def update_medical_condition(selected_gender):
    filtered_data=data[data['Gender']== selected_gender] if selected_gender else data
    fig=px.pie(filtered_data,names='Medical Condition',title='Medical Condition Distribution')
    return fig

# 3.callback for Insurance Provider Comparison
@app.callback(
    Output("insurance-comparison",'figure'),
    Input("gender-filter","value")
)
def update_insurance(selected_gender):
    filter_data=data[data['Gender'] == selected_gender] if selected_gender else data
    fig=px.bar(
        filter_data,
        x='Insurance Provider',
        y='Billing Amount',
        color='Medical Condition',
        barmode='group',
        title='Insurance Provider Price Comparison',
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    return fig

#3. Callback for Billing Distribution
@app.callback(
    Output('billing-distribution','figure'),
    [Input('gender-filter','value'),Input('billing-slider','value')]
)
def update_insurance(selected_gender,slider_value):
    filter_data=data[data['Gender'] == selected_gender] if selected_gender else data
    filter_data=filter_data[filter_data['Billing Amount']<= slider_value]
    fig=px.histogram(filter_data,
                     x='Billing Amount',
                     nbins=10,
                     title='Billing Amount Distribution')
    return fig

# Trend in Admission
@app.callback(
    Output('admission-trends','figure'),
    [Input('chart-type','value'),Input('condition-filter','value')]
)

def update_admission(chart_type,selected_condition):
    filter_data=data[data['Medical Condition']==selected_condition] if selected_condition else data
    trend_data=filter_data.groupby('YearMonth').size().reset_index(name='Count')
    trend_data['YearMonth']=trend_data['YearMonth'].astype(str)

    if chart_type== 'line':
        fig=px.line(trend_data,x='YearMonth',y='Count',title='Admission Trends over Time')
    else:
        fig=px.bar(trend_data,x='YearMonth',y='Count',title='Admission Trends over Time')
    return fig
app.run(debug=True)

#if __name__=="__main__":
 # app.run_server(debug=True)
